# Load Embeddings Directly Into Rust From Parquet

Demonstrates `NNdex.from_file(..., key="embedding")` plus the new `search(query, k, dataframe=...)` API.


In [1]:
from pathlib import Path
import numpy as np
import polars as pl
from nndex import NNdex


In [2]:
PARQUET_PATH = Path("/Users/maxwoolf/Desktop/mtg-embeddings/mtg_embeddings.parquet")
assert PARQUET_PATH.exists(), f"Missing parquet file: {PARQUET_PATH}"

schema = pl.read_parquet_schema(PARQUET_PATH)
columns = list(schema.keys())
print("Columns:", columns)
assert "embedding" in columns, "Expected `embedding` column in parquet schema."


Columns: ['name', 'scryfallId', 'manaCost', 'type', 'text', 'power', 'toughness', 'loyalty', 'rarities', 'sets', 'embedding', 'x_2d', 'y_2d']


In [3]:
# 1) Load embeddings directly into Rust.
index = NNdex.from_file(str(PARQUET_PATH), key="embedding", normalized=False, backend="cpu", enable_cache=False)
print("NNdex rows:", index.rows, "dims:", index.dims, "backend:", index.backend)


NNdex rows: 32254 dims: 768 backend: cpu


In [4]:
# 2) Load metadata in Polars with all columns except embedding.
metadata_columns = [column for column in columns if column != "embedding"]
metadata_df = pl.read_parquet(PARQUET_PATH, columns=metadata_columns)
assert metadata_df.height == index.rows
metadata_df.head(3)


shape: (3, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ name      ┆ scryfallI ┆ manaCost  ┆ type      ┆ … ┆ rarities  ┆ sets      ┆ x_2d      ┆ y_2d     │
│ ---       ┆ d         ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ str       ┆ str       ┆   ┆ list[enum ┆ list[enum ┆ f32       ┆ f32      │
│           ┆ str       ┆           ┆           ┆   ┆ ]         ┆ ]         ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ "Ach!     ┆ 84f2c8f5- ┆ {2}{R}{R} ┆ Enchantme ┆ … ┆ ["rare"]  ┆ ["UNH"]   ┆ -0.261871 ┆ 2.217163 │
│ Hans,     ┆ 8e11-4639 ┆ {G}{G}    ┆ nt        ┆   ┆           ┆           ┆           ┆          │
│ Run!"     ┆ -b7de-00e ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ 4a2…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ "Brims"   ┆ 68832214- ┆ {3}{W}{B} ┆ Legendary ┆ … ┆ ["uncommo ┆ ["UNF"]   ┆ 0.045746  ┆ -0.99035 │
│ Barone,   ┆ 2943-4253 ┆           ┆ Creature  ┆   ┆ n"]       ┆           ┆           ┆ 6        │
│ Midway    ┆ -8884-ffa ┆           ┆ — Human   ┆   ┆           ┆           ┆           ┆          │
│ Mobster   ┆ 490…      ┆           ┆ Rog…      ┆   ┆           ┆           ┆           ┆          │
│ "Lifetime ┆ 42293306- ┆ {B}       ┆ Creature  ┆ … ┆ ["rare"]  ┆ ["UNF"]   ┆ -1.034943 ┆ 0.165527 │
│ " Pass    ┆ aaea-4542 ┆           ┆ — Zombie  ┆   ┆           ┆           ┆           ┆          │
│ Holder    ┆ -8df4-813 ┆           ┆ Guest     ┆   ┆           ┆           ┆           ┆          │
│           ┆ 823…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [5]:
# Build query matrix from embeddings for prediction tests.
embedding_df = pl.read_parquet(PARQUET_PATH, columns=["embedding"], n_rows=6)
query_matrix = np.asarray(embedding_df.get_column("embedding").to_list(), dtype=np.float32)
query_matrix.shape


(6, 768)

In [7]:
# 3a) Prediction with DataFrame output by passing dataframe=...
# Use the polars metadata dataframe directly.
results = index.search(query_matrix[:3], k=3, dataframe=metadata_df)
type(results), len(results), results


(list,
 3,
 [shape: (3, 13)
  ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
  │ name      ┆ scryfallI ┆ manaCost  ┆ type      ┆ … ┆ sets      ┆ x_2d      ┆ y_2d     ┆ similarit │
  │ ---       ┆ d         ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---      ┆ y         │
  │ str       ┆ ---       ┆ str       ┆ str       ┆   ┆ list[enum ┆ f32       ┆ f32      ┆ ---       │
  │           ┆ str       ┆           ┆           ┆   ┆ ]         ┆           ┆          ┆ f64       │
  ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
  │ "Ach!     ┆ 84f2c8f5- ┆ {2}{R}{R} ┆ Enchantme ┆ … ┆ ["UNH"]   ┆ -0.261871 ┆ 2.217163 ┆ 1.0       │
  │ Hans,     ┆ 8e11-4639 ┆ {G}{G}    ┆ nt        ┆   ┆           ┆           ┆          ┆           │
  │ Run!"     ┆ -b7de-00e ┆           ┆           ┆   ┆           ┆           ┆          ┆           │
  │           ┆ 4a2…      ┆           ┆      

In [8]:
# 3b) Default behavior without dataframe=... remains numpy tuple output.
indices, scores = index.search(query_matrix[:3], k=3)
indices.shape, scores.shape


((3, 3), (3, 3))

In [9]:
# 3c) Row-count mismatch validation for dataframe=...
try:
    index.search(query_matrix[0], k=3, dataframe=metadata_df[:5])
    raise AssertionError("Expected ValueError for dataframe row mismatch")
except ValueError as exc:
    print("Expected error:", exc)
    assert "row count must match index rows" in str(exc)


Expected error: provided dataframe row count must match index rows (index rows=32254, dataframe rows=5)
